<a href="https://colab.research.google.com/github/ApurvVats/Netflix_clone/blob/main/Web_Content_Extraction_Analysis_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing library
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re
from google.colab import files
import csv


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

from google.colab import drive
# drive.mount('/gdrive', force_remount=True)
%cd '/gdrive/MyDrive/20211030 Test Assignment'

/gdrive/MyDrive/20211030 Test Assignment


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/20211030 Test Assignment/Input.xlsx'

In [ ]:
df = pd.read_excel(file_path)[['URL_ID', 'URL']]

In [ ]:
df=df.iloc[0:150]

In [ ]:
df.head()

,URL_ID,URL
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...
3,bctech2014,https://insights.blackcoffer.com/effective-man...
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...


In [ ]:
for index, row in df.iterrows():
  url = row['URL']
  url_id = row['URL_ID']

  # make a request to url
  header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
  try:
    response = requests.get(url,headers=header)
  except:
    print("can't get response of {}".format(url_id))

  #create a beautifulsoup object
  try:
    soup = BeautifulSoup(response.content, 'html.parser')
  except:
    print("can't get page of {}".format(url_id))
  #find title
  try:
    title = soup.find('h1').get_text()
  except:
    print("can't get title of {}".format(url_id))
    continue
  #find text
  article = ""
  try:
    for p in soup.find_all('p'):
      article += p.get_text()
  except:
    print("can't get text of {}".format(url_id))

  file_name = '/content/drive/MyDrive/20211030 Test Assignment/'+str(url_id)+'.txt'
  with open(file_name, 'w') as file:
    file.write(title + '\n' + article)

In [ ]:
text_dic = "/content/drive/MyDrive/20211030 Test Assignment"
stopwords_dir = "/content/drive/My Drive/20211030 Test Assignment/StopWords"
sentment_dir = "/content/drive/My Drive/20211030 Test Assignment/MasterDictionary"

stop_words = set()
for files in os.listdir(stopwords_dir):
  with open(os.path.join(stopwords_dir,files),'r',encoding='ISO-8859-1') as f:
    stop_words.update(set(f.read().splitlines()))

# load all text files  from the  directory and store in a list(docs)
docs = []
for text_file in os.listdir(text_dic):
  # Check if the file is a text file
  if text_file.endswith(".txt"):
    with open(os.path.join(text_dic,text_file),'r', encoding='ISO-8859-1') as f:
      text = f.read()
    #tokenize the given text file
      words = word_tokenize(text)
    # remove the stop words from the tokens
      filtered_text = [word for word in words if word.lower() not in stop_words]
    # add each filtered tokens of each file into a list
      docs.append(filtered_text)

#tokenize the given text file
    words = word_tokenize(text)
# remove the stop words from the tokens
    filtered_text = [word for word in words if word.lower() not in stop_words]
# add each filtered tokens of each file into a list
    docs.append(filtered_text)



# store positive, Negative words from the directory
pos=set()
neg=set()

for files in os.listdir(sentment_dir):
  if files =='positive-words.txt':
    with open(os.path.join(sentment_dir,files),'r',encoding='ISO-8859-1') as f:
      pos.update(f.read().splitlines())
  else:
    with open(os.path.join(sentment_dir,files),'r',encoding='ISO-8859-1') as f:
      neg.update(f.read().splitlines())

# now collect the positive  and negative words from each file
# calculate the scores from the positive and negative words
positive_words = []
Negative_words =[]
positive_score = []
negative_score = []
polarity_score = []
subjectivity_score = []

#iterate through the list of docs
for i in range(len(docs)):
  positive_words.append([word for word in docs[i] if word.lower() in pos])
  Negative_words.append([word for word in docs[i] if word.lower() in neg])
  positive_score.append(len(positive_words[i]))
  negative_score.append(len(Negative_words[i]))
  polarity_score.append((positive_score[i] - negative_score[i]) / ((positive_score[i] + negative_score[i]) + 0.000001))
  subjectivity_score.append((positive_score[i] + negative_score[i]) / ((len(docs[i])) + 0.000001))



In [ ]:
# Average Sentence Length = the number of words / the number of sentences
# Percentage of Complex words = the number of complex words / the number of words
# Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

import os
import re
from nltk.corpus import stopwords

# Define your lists to store results
avg_sentence_length = []
Percentage_of_Complex_words = []
Fog_Index = []
complex_word_count = []
avg_syllable_word_count = []

# Initialize stop words
stop_words = set(stopwords.words('english'))

def measure(file):
    file_path = os.path.join(text_dic, file)

    # Check if it's a file and not a directory
    if os.path.isfile(file_path):
        try:
            # Read the file with ISO-8859-1 encoding to avoid decoding errors
            with open(file_path, 'r', encoding='ISO-8859-1') as f:
                text = f.read()

            # Remove punctuations
            text = re.sub(r'[^\w\s.]', '', text)

            # Split the text into sentences
            sentences = text.split('.')

            # Total number of sentences
            num_sentences = len(sentences)

            # Filter out stop words and count total words
            words = [word for word in text.split() if word.lower() not in stop_words]
            num_words = len(words)

            # Complex words (more than 2 syllables)
            complex_words = []
            vowels = 'aeiou'
            for word in words:
                syllable_count_word = sum(1 for letter in word if letter.lower() in vowels)
                if syllable_count_word > 2:
                    complex_words.append(word)

            # Syllable count per word
            syllable_count = 0
            syllable_words = []
            for word in words:
                if word.endswith('es'):
                    word = word[:-2]
                elif word.endswith('ed'):
                    word = word[:-2]
                syllable_count_word = sum(1 for letter in word if letter.lower() in vowels)
                if syllable_count_word >= 1:
                    syllable_words.append(word)
                    syllable_count += syllable_count_word

            # Calculate metrics
            avg_sentence_len = num_words / num_sentences if num_sentences > 0 else 0
            avg_syllable_word_count = syllable_count / len(syllable_words) if len(syllable_words) > 0 else 0
            Percent_Complex_words = len(complex_words) / num_words if num_words > 0 else 0
            Fog_Index = 0.4 * (avg_sentence_len + Percent_Complex_words)

            return avg_sentence_len, Percent_Complex_words, Fog_Index, len(complex_words), avg_syllable_word_count

        except Exception as e:
            print(f"An error occurred while reading {file}: {e}")
            return None, None, None, None, None
    else:
        print(f"Skipping {file_path} as it is not a file.")
        return None, None, None, None, None

# Iterate through each file in the directory
for file in os.listdir(text_dic):
    result = measure(file)
    if result and all(result):  # Only append if result is valid
        x, y, z, a, b = result
        avg_sentence_length.append(x)
        Percentage_of_Complex_words.append(y)
        Fog_Index.append(z)
        complex_word_count.append(a)
        avg_syllable_word_count.append(b)


Skipping /content/drive/MyDrive/20211030 Test Assignment/StopWords as it is not a file.
Skipping /content/drive/MyDrive/20211030 Test Assignment/MasterDictionary as it is not a file.


In [ ]:
import os
import re
from nltk.corpus import stopwords

# Ensure nltk stopwords are downloaded
import nltk
nltk.download('stopwords')

# Set stopwords
stop_words = set(stopwords.words('english'))

def cleaned_words(file):
    file_path = os.path.join(text_dic, file)

    # Check if it's a file before attempting to read it
    if os.path.isfile(file_path):
        try:
            # Open the file with ISO-8859-1 encoding to avoid decoding errors
            with open(file_path, 'r', encoding='ISO-8859-1') as f:
                text = f.read()
                # Remove punctuations
                text = re.sub(r'[^\w\s]', '', text)
                # Filter out stopwords
                words = [word for word in text.split() if word.lower() not in stop_words]
                # Calculate total word length and average word length
                length = sum(len(word) for word in words)
                average_word_length = length / len(words) if words else 0
            return len(words), average_word_length
        except Exception as e:
            print(f"An error occurred while processing {file}: {e}")
            return 0, 0  # Return 0 if an error occurs
    else:
        print(f"Skipping {file_path} as it is not a file.")
        return 0, 0

# Initialize lists for storing results
word_count = []
average_word_length = []

# Iterate through each file in the directory
for file in os.listdir(text_dic):
    x, y = cleaned_words(file)
    word_count.append(x)
    average_word_length.append(y)

# Output the results
print("Word Count:", word_count)
print("Average Word Length:", average_word_length)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Skipping /content/drive/MyDrive/20211030 Test Assignment/StopWords as it is not a file.
Skipping /content/drive/MyDrive/20211030 Test Assignment/MasterDictionary as it is not a file.
Word Count: [694, 0, 0, 1071, 504, 527, 634, 205, 205, 206, 206, 203, 206, 206, 212, 206, 209, 514, 401, 294, 506, 453, 444, 543, 487, 516, 566, 294, 402, 350, 415, 541, 650, 326, 293, 259, 193, 245, 302, 388, 308, 242, 300, 269, 391, 308, 388, 292, 312, 343, 385, 377, 304, 226, 213, 279, 289, 336, 230, 345, 390, 271, 360, 374, 250, 208, 220, 228, 214, 251, 326, 364, 376, 412, 371, 335, 601, 316, 354, 192, 329, 316, 222, 256, 303, 360, 192, 250, 316, 340, 283, 201, 243, 295, 252, 237, 272, 605, 628, 190, 311, 246, 313, 280, 353, 278, 333, 214, 310, 165, 500, 337, 338, 234, 313, 336, 428, 312, 356, 192, 255, 224, 360, 309, 160, 203, 213, 202, 193, 189, 316, 232, 381, 275, 348, 306, 324, 312, 231, 230, 219, 225, 224, 207, 210, 278, 711, 517, 705, 1008, 265, 161, 212]
Average Word Length: [14.802593659942364,

In [ ]:
import os
import re

def count_personal_pronouns(file):
    file_path = os.path.join(text_dic, file)

    # Check if it's a file before attempting to read it
    if os.path.isfile(file_path):
        try:
            # Open the file with ISO-8859-1 encoding to avoid decoding errors
            with open(file_path, 'r', encoding='ISO-8859-1') as f:
                text = f.read()
                # Define personal pronouns
                personal_pronouns = ["I", "we", "my", "ours", "us"]
                count = 0
                # Count occurrences of each personal pronoun
                for pronoun in personal_pronouns:
                    count += len(re.findall(r"\b" + re.escape(pronoun) + r"\b", text))
            return count
        except Exception as e:
            print(f"An error occurred while processing {file}: {e}")
            return 0  # Return 0 if an error occurs
    else:
        print(f"Skipping {file_path} as it is not a file.")
        return 0

# Initialize list for storing results
pp_count = []

# Iterate through each file in the directory
for file in os.listdir(text_dic):
    x = count_personal_pronouns(file)
    pp_count.append(x)

# Output the results
print("Personal Pronoun Counts:", pp_count)


Skipping /content/drive/MyDrive/20211030 Test Assignment/StopWords as it is not a file.
Skipping /content/drive/MyDrive/20211030 Test Assignment/MasterDictionary as it is not a file.
Personal Pronoun Counts: [6, 0, 0, 11, 19, 12, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 3, 12, 7, 4, 2, 14, 5, 10, 4, 3, 2, 6, 2, 11, 7, 3, 6, 2, 4, 2, 7, 4, 4, 2, 3, 10, 3, 2, 5, 10, 16, 5, 2, 2, 2, 3, 2, 2, 9, 2, 4, 4, 3, 6, 3, 2, 3, 2, 3, 2, 3, 7, 7, 5, 8, 10, 4, 5, 2, 2, 2, 4, 4, 2, 1, 6, 1, 1, 1, 2, 1, 1, 2, 1, 3, 2, 2, 1, 2, 1, 1, 2, 1, 1, 4, 5, 4, 4, 3, 4, 1, 2, 2, 1, 1, 3, 6, 7, 3, 2, 3, 1, 2, 3, 4, 2, 1, 1, 1, 1, 1, 4, 3, 4, 2, 7, 4, 1, 7, 1, 1, 1, 1, 1, 1, 1, 3, 6, 2, 3, 5, 10, 1, 2]


In [ ]:
data={'positive_score':positive_score,'negative_score':negative_score,'Polarity_Score':polarity_score,'subjectivity_score':subjectivity_score,'avg_senetence_length':avg_sentence_length,'Percentage_of_Complex_words':Percentage_of_Complex_words,'Fog_Index':Fog_Index,'avg_no_of_words_per_sentence':avg_sentence_length,'complex_Word_Count':complex_word_count,'word_count':word_count,'syllable_count':avg_syllable_word_count,'personal_pronouns':pp_count,'avg_word_length':average_word_length}


In [ ]:
new = pd.DataFrame.from_dict(data)

new
new.to_csv('file1.csv')